In [1]:
# Run this cell to install pyarrow
!pip install pyarrow

In [ ]:
# #CONVERT CSV TO PARQUET FOR MORE EFFICIENT NN TRAINING

# import pandas as pd
# import pyarrow as pa
# import pyarrow.parquet as pq

# # Define data types explicitly for robust CSV reading
# dtype_spec = {
#     'FG3_PCT': float,
#     'FT_PCT': float,
#     # Define other columns with their correct data types
# }

# # Define paths for your CSV and output Parquet file
# csv_file = '/Users/byrondowdell/Library/CloudStorage/OneDrive-Personal/Grad School/Spring 2024/Advanced Machine Learning/Project/Sports Betting/10%rand_samp_player_prop_final_dataset.csv'
# parquet_file = '/Users/byrondowdell/Library/CloudStorage/OneDrive-Personal/Grad School/Spring 2024/Advanced Machine Learning/Project/Sports Betting/10%rand_samp_player_prop_final_dataset.parquet'

# # Process the CSV in chunks
# csv_stream = pd.read_csv(csv_file, chunksize=100000, dtype=dtype_spec)
# writer = None
# parquet_schema = None

# for chunk in csv_stream:
#     # Handle nulls if present
#     chunk.fillna({
#         'FG3_PCT': 0.0,
#         'FT_PCT': 0.0,
#     }, inplace=True)

#     # Convert DataFrame to Table
#     table = pa.Table.from_pandas(chunk, schema=parquet_schema, preserve_index=False)

#     if parquet_schema is None:
#         parquet_schema = table.schema
#         writer = pq.ParquetWriter(parquet_file, parquet_schema, compression='snappy')

#     writer.write_table(table)

# if writer:
#     writer.close()


/var/folders/hj/z1s1lgxn7r5gsnhffrk8zl_80000gn/T/ipykernel_1687/3429619649.py:23: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in csv_stream:


In [11]:
import pandas as pd
# Path to your Parquet file
NBA_parquet_file = "10%rand_samp_player_prop_final_dataset.parquet"

# Read the Parquet file
NBA_df = pd.read_parquet(NBA_parquet_file)

NBA_df.head(2)

,id,away_team,home_team,GAME_DATE,Player_ID,player_name,Game_ID,prop_market,prop_type,prop,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,Bet_outcome
0,5a4b28938902c4fba16da19c6e8c1296,Milwaukee Bucks,New York Knicks,2023-12-25,1626171,Bobby Portis,22300401.0,player_assists,Under,0.5,...,4,9,0,0,0,3,4,12,-3,0
1,5a4b28938902c4fba16da19c6e8c1296,Milwaukee Bucks,New York Knicks,2023-12-25,1626171,Bobby Portis,22300401.0,player_assists,Under,0.5,...,4,9,0,0,0,3,4,12,-3,0


In [12]:
NBA_df.columns

Index(['id', 'away_team', 'home_team', 'GAME_DATE', 'Player_ID', 'player_name',
       'Game_ID', 'prop_market', 'prop_type', 'prop', 'WL', 'MIN', 'FGM',
       'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'PLUS_MINUS', 'Bet_outcome'],
      dtype='object')

In [ ]:
# #NBA_df["WL"]

# import pandas as pd


# # Replace 'W' with 1 and 'L' with 0 in the 'WL' column
# NBA_df['WL'] = NBA_df['WL'].replace({'W': 1, 'L': 0})

# # Optionally, check the transformation's correctness
# NBA_df.head(2)


In [13]:
NBA_df["WL"].value_counts()
#class imbalance isnt too bad!

WL
0    6642554
1    5658256
Name: count, dtype: int64

In [14]:
NBA_df.shape

(12300810, 32)

In [15]:
!pip install tensorflow

In [16]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping  # Import EarlyStopping

# Encode player names to integer indices
label_encoder = LabelEncoder()
player_indices = label_encoder.fit_transform(NBA_df['player_name'])

# Determine the number of unique players
num_players = len(label_encoder.classes_)

# Prepare the target variable
target = NBA_df['Bet_outcome'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(player_indices, target, test_size=0.2, random_state=42)

# Reshape the input data for embedding layer compatibility
X_train = X_train.reshape(-1, 1)
X_test = X_test.reshape(-1, 1)

# Define the neural network model with an embedding layer
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=num_players, output_dim=50, input_length=1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model with appropriate loss function and optimizer
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Configure Early Stopping
early_stopping_monitor = EarlyStopping(
    monitor='val_loss',     # Monitor validation loss
    patience=3,             # How many epochs to wait after min has been hit
    verbose=1,              # Verbosity mode
    mode='min',             # The direction is "minimizing" for loss
    restore_best_weights=True  # Restores model weights from the epoch with the best value of the monitored quantity.
)

# Train the model with the EarlyStopping callback
model.fit(X_train, y_train, epochs=10, batch_size=256, validation_split=0.2, callbacks=[early_stopping_monitor])

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.2f}')


Epoch 1/10
30753/30753 [==============================] - 120s 4ms/step - loss: 0.6888 - accuracy: 0.5295 - val_loss: 0.6886 - val_accuracy: 0.5301
Epoch 2/10
30753/30753 [==============================] - 111s 4ms/step - loss: 0.6887 - accuracy: 0.5298 - val_loss: 0.6886 - val_accuracy: 0.5297
Epoch 3/10
30753/30753 [==============================] - 111s 4ms/step - loss: 0.6887 - accuracy: 0.5295 - val_loss: 0.6887 - val_accuracy: 0.5294
Epoch 4/10
30753/30753 [==============================] - 112s 4ms/step - loss: 0.6887 - accuracy: 0.5292 - val_loss: 0.6886 - val_accuracy: 0.5296
Epoch 5/10
30753/30753 [==============================] - 111s 4ms/step - loss: 0.6887 - accuracy: 0.5291 - val_loss: 0.6886 - val_accuracy: 0.5296
Epoch 6/10
30753/30753 [==============================] - 111s 4ms/step - loss: 0.6887 - accuracy: 0.5290 - val_loss: 0.6886 - val_accuracy: 0.5296
Epoch 7/10
30753/30753 [==============================] - 111s 4ms/step - loss: 0.6887 - accuracy: 0.5290 - val_

In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping  # Import EarlyStopping

# Encode player names to integer indices
label_encoder = LabelEncoder()
player_indices = label_encoder.fit_transform(NBA_df['player_name'])

# Determine the number of unique players
num_players = len(label_encoder.classes_)

# Prepare the target variable
target = NBA_df['Bet_outcome'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(player_indices, target, test_size=0.2, random_state=42)

# Reshape the input data for embedding layer compatibility
X_train = X_train.reshape(-1, 1)
X_test = X_test.reshape(-1, 1)

# Define the neural network model with an embedding layer
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=num_players, output_dim=50, input_length=1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model with appropriate loss function and optimizer
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Configure Early Stopping
early_stopping_monitor = EarlyStopping(
    monitor='val_loss',     # Monitor validation loss
    patience=3,             # How many epochs to wait after min has been hit
    verbose=1,              # Verbosity mode
    mode='min',             # The direction is "minimizing" for loss
    restore_best_weights=True  # Restores model weights from the epoch with the best value of the monitored quantity.
)

# Train the model with the EarlyStopping callback
model.fit(X_train, y_train, epochs=20, batch_size=128, validation_split=0.2, callbacks=[early_stopping_monitor])

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.2f}')


Epoch 1/20
61505/61505 [==============================] - 231s 4ms/step - loss: 0.6888 - accuracy: 0.5296 - val_loss: 0.6887 - val_accuracy: 0.5293
Epoch 2/20
61505/61505 [==============================] - 224s 4ms/step - loss: 0.6887 - accuracy: 0.5295 - val_loss: 0.6886 - val_accuracy: 0.5297
Epoch 3/20
61505/61505 [==============================] - 223s 4ms/step - loss: 0.6887 - accuracy: 0.5292 - val_loss: 0.6887 - val_accuracy: 0.5291
Epoch 4/20
61505/61505 [==============================] - 224s 4ms/step - loss: 0.6887 - accuracy: 0.5289 - val_loss: 0.6886 - val_accuracy: 0.5296
Epoch 5/20
61505/61505 [==============================] - 223s 4ms/step - loss: 0.6887 - accuracy: 0.5290 - val_loss: 0.6886 - val_accuracy: 0.5296
Epoch 6/20
61505/61505 [==============================] - 222s 4ms/step - loss: 0.6887 - accuracy: 0.5289 - val_loss: 0.6887 - val_accuracy: 0.5269
Epoch 7/20
61505/61505 [==============================] - 221s 4ms/step - loss: 0.6887 - accuracy: 0.5288 - val_